# How much does the image position vary due to hexapod motions?

Craig Lage  19-Aug-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial import polynomial as npp
from lsst.daf.butler import Butler
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData, calcNextDay, getMostRecentRowWithDataBefore
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from astropy.time import Time, TimeDelta
import yaml

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])

In [ ]:
lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/laser_rotation_elevation_v20_positive.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data_1 = yaml.safe_load(yaml_file)

lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/laser_rotation_elevation_v20_negative.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data_2 = yaml.safe_load(yaml_file)



In [ ]:
lut_data_2

In [ ]:

el = np.linspace(20, 85, 14)
for r in [-45, 0, 45]:
    rot = np.linspace(r, r, 14)
    i = 1
    lut_coeffs_4 = np.array(lut_data_4['camera_config']['elevation_rotation_coeffs'][i])
    lut_coeffs_3 = lut_data_3['camera_config']['elevation_coeffs'][i]
    #lut_coeffs_1
    #lut = np.polyval(lut_coeffs_3[::-1], el)
    lut = npp.polyval2d(el, rot, lut_coeffs_4)
    plt.plot(el, lut, marker='x', label=f"Rot={r:.0f}")
plt.legend()

In [ ]:
lut_coeffs_4

In [ ]:
startDay = 20250903
endDay = 20250903

dayObs = startDay
expIds = []
xaxis = []
camPoss = []
camLUTs = []
m2Poss = []
m2LUTs = []
temps = []
counter = 1
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.id < 2025090300157:
            continue
        if record.observation_type not in ['acq', 'science']:
            continue
        try:
            """
            temp = getEfdData(client, 'lsst.sal.ESS.temperature',
                 expRecord=record)
            m2Temp = temp[temp['sensorName']=='M2-ESS02']
            strutTemp = (np.nanmedian(m2Temp['temperatureItem6'].values) + \
                          np.nanmedian(m2Temp['temperatureItem7'].values)) / 2.0
            temps.append(strutTemp) 
            """
            el = 90.0 - record.zenith_angle
            (mountErrors, mountData) = calculateMountErrors(record, client)
            rotVals = mountData.rotationData['actualPosition'].values
            rot = (np.median(rotVals))
            poss = []
            luts = []
            for i in range(5):
                val = np.median(mountData.camhexData[f"position{i}"].values)
                lut_coeffs = lut_data_3['camera_config']['elevation_coeffs'][i]
                lut = np.polyval(lut_coeffs[::-1], el)
                rot_coeffs = lut_data_1['camera_config']['rotation_coeffs'][i]
                lut += np.polyval(rot_coeffs[::-1], rot)
                if i in [3, 4]:
                    val *= 3600.0
                    lut *= 3600.0
                poss.append(val)
                luts.append(lut)
            camPoss.append(poss)
            camLUTs.append(luts)
            poss = []
            luts = []
            for i in range(5):
                val = np.median(mountData.m2hexData[f"position{i}"].values)
                lut_coeffs = lut_data_3['m2_config']['elevation_coeffs'][i]
                lut = np.polyval(lut_coeffs[::-1], el)
                rot_coeffs = lut_data_1['m2_config']['rotation_coeffs'][i]
                lut += np.polyval(rot_coeffs[::-1], rot)
                if i in [3, 4]:
                    val *= 3600.0
                    lut *= 3600.0
                poss.append(val)
                luts.append(lut)
            m2Poss.append(poss)
            m2LUTs.append(luts)
            xaxis.append(counter)
            expIds.append(record.id)
            counter += 1
            print(f"{record.id} succeeded!")
        except:
            print(f"{record.id} failed!")
            continue
    dayObs = calcNextDay(dayObs)


In [ ]:
dayObs = startDay
names = ['X', 'Y', 'Z', 'U', 'V']
arrExpIds = np.array(expIds)
seqNums = (arrExpIds - (arrExpIds/1E5).astype(int) * 1E5).astype(int)
fig, axs = plt.subplots(2, 5, figsize=(16, 5))
plt.subplots_adjust(wspace=0.8, hspace=0.6)
plt.suptitle(f"Hexapod changes vs LUT {startDay}-{endDay}", fontsize=18, y=1.0)
for i in range(5):
    cam = [term[i] for term in camPoss]
    lut = [term[i] for term in camLUTs]
    axs[0][i].scatter(xaxis, cam, marker='x')
    axs[0][i].scatter(xaxis, lut, marker='.', color='red')
    axs[0][i].set_xticks(xaxis[::50])
    axs[0][i].set_xticklabels(seqNums[::50], fontsize=10)
    axs[0][i].tick_params(axis='x', labelrotation=90)
    axs[0][i].set_title(f'Cam{names[i]}')
    #axs[0][i].axvline(239.5, ls='--', color='green')
    if i < 3:
        axs[0][i].set_ylabel("Microns")
    else:
        axs[0][i].set_ylabel("Arcseconds")
    m2 = [term[i] for term in m2Poss]
    lut = [term[i] for term in m2LUTs]
    axs[1][i].scatter(xaxis, m2, marker='x')
    axs[1][i].scatter(xaxis, lut, marker='.', color='red')
    axs[1][i].set_xticks(xaxis[::50])
    axs[1][i].set_xticklabels(seqNums[::50], fontsize=10)
    axs[1][i].tick_params(axis='x', labelrotation=90)
    axs[1][i].set_title(f'M2{names[i]}')
    #axs[1][i].axvline(239.5, ls='--', color='green')
    if i < 3:
        axs[1][i].set_ylabel("Microns")
    else:
        axs[1][i].set_ylabel("Arcseconds")
plt.savefig(f"/home/c/cslage/u/Hexapods/LUTs/Hexapod_Changes_LUT_{startDay}.png")
plt.clf


## I want to check if the values in lsst.sal.MTHexapod.logevent_compensationOffset match the calculated LUT values

In [ ]:
startDay = 20250827
endDay = 20250827

names = ['x', 'y', 'z', 'u', 'v']
dayObs = startDay
expIds = []
xaxis = []
camPoss = []
camComps = []
camLUTs = []
m2Poss = []
m2Comps = []
m2LUTs = []
temps = []
els[0:240] = []
rots = []
counter = 1

def CamSalIndexTest(df):
    return df['salIndex'] == 1
def M2SalIndexTest(df):
    return df['salIndex'] == 2
    
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['acq', 'science']:
            continue
        try:
            if record.id < 2025082700367:
                lut_data = lut_data_2
            else:
                lut_data = lut_data_1
            temp = getEfdData(client, 'lsst.sal.ESS.temperature',
                 expRecord=record)
            m2Temp = temp[temp['sensorName']=='M2-ESS02']
            strutTemp = (np.nanmedian(m2Temp['temperatureItem6'].values) + \
                          np.nanmedian(m2Temp['temperatureItem7'].values)) / 2.0
            temps.append(strutTemp)  
            el = 90.0 - record.zenith_angle
            (mountErrors, mountData) = calculateMountErrors(record, client)
            rotVals = mountData.rotationData['actualPosition'].values
            rot = (np.median(rotVals))
            poss = []
            luts = []
            comps = []
            comp = getMostRecentRowWithDataBefore(client, "lsst.sal.MTHexapod.logevent_compensationOffset", \
                                                    record.timespan.begin, where=CamSalIndexTest)
            for i in range(5):
                val = np.median(mountData.camhexData[f"position{i}"].values)
                lut_coeffs = lut_data['camera_config']['elevation_coeffs'][i]
                lut = np.polyval(lut_coeffs[::-1], el)
                rot_coeffs = lut_data_1['camera_config']['rotation_coeffs'][i]
                lut += np.polyval(rot_coeffs[::-1], rot)
                thisComp = comp[names[i]]
                if i in [3, 4]:
                    val *= 3600.0
                    lut *= 3600.0
                    thisComp *= 3600.0
                poss.append(val)
                luts.append(lut)
                comps.append(thisComp)
            camPoss.append(poss)
            camLUTs.append(luts)
            camComps.append(comps)
            poss = []
            luts = []
            comps = []
            comp = getMostRecentRowWithDataBefore(client, "lsst.sal.MTHexapod.logevent_compensationOffset", \
                                                    record.timespan.begin, where=M2SalIndexTest)
            for i in range(5):
                val = np.median(mountData.m2hexData[f"position{i}"].values)
                lut_coeffs = lut_data['m2_config']['elevation_coeffs'][i]
                lut = np.polyval(lut_coeffs[::-1], el)
                rot_coeffs = lut_data_1['m2_config']['rotation_coeffs'][i]
                lut += np.polyval(rot_coeffs[::-1], rot)
                thisComp = comp[names[i]]
                if i in [3, 4]:
                    val *= 3600.0
                    lut *= 3600.0
                    thisComp *= 3600.0
                poss.append(val)
                luts.append(lut)
                comps.append(thisComp)
            m2Poss.append(poss)
            m2LUTs.append(luts)
            m2Comps.append(comps)
            xaxis.append(counter)
            expIds.append(record.id)
            els.append(el)
            rots.append(rot)

            counter += 1
            print(f"{record.id} succeeded!")
        except:
            print(f"{record.id} failed!")
            continue
    dayObs = calcNextDay(dayObs)


In [ ]:
dayObs = startDay
names = ['X', 'Y', 'Z', 'U', 'V']
arrExpIds = np.array(expIds)
seqNums = (arrExpIds - (arrExpIds/1E5).astype(int) * 1E5).astype(int)
fig, axs = plt.subplots(2, 5, figsize=(16, 5))
plt.subplots_adjust(wspace=0.8, hspace=0.6)
plt.suptitle(f"Compensation Offsets vs LUT {startDay}-{endDay}", fontsize=18, y=1.0)
for i in range(5):
    comps = [term[i] for term in camComps]
    lut = [term[i] for term in camLUTs]
    axs[0][i].scatter(xaxis, comps, marker='x')
    axs[0][i].scatter(xaxis, lut, marker='.', color='red')
    axs[0][i].set_xticks(xaxis[::50])
    axs[0][i].set_xticklabels(seqNums[::50], fontsize=10)
    axs[0][i].tick_params(axis='x', labelrotation=90)
    axs[0][i].set_title(f'Cam{names[i]}')
    axs[0][i].axvline(239.5, ls='--', color='green')
    if i < 3:
        axs[0][i].set_ylabel("Microns")
    else:
        axs[0][i].set_ylabel("Arcseconds")
    comps = [term[i] for term in m2Comps]
    lut = [term[i] for term in m2LUTs]
    axs[1][i].scatter(xaxis, comps, marker='x')
    axs[1][i].scatter(xaxis, lut, marker='.', color='red')
    axs[1][i].set_xticks(xaxis[::50])
    axs[1][i].set_xticklabels(seqNums[::50], fontsize=10)
    axs[1][i].tick_params(axis='x', labelrotation=90)
    axs[1][i].set_title(f'M2{names[i]}')
    axs[1][i].axvline(239.5, ls='--', color='green')
    if i < 3:
        axs[1][i].set_ylabel("Microns")
    else:
        axs[1][i].set_ylabel("Arcseconds")
plt.savefig(f"/home/c/cslage/u/Hexapods/LUTs/LUT_vs_CompOffset_{startDay}-{endDay}.png")
plt.clf


In [ ]:
dayObs = startDay
names = ['X', 'Y', 'Z', 'U', 'V']
arrExpIds = np.array(expIds)
seqNums = (arrExpIds - (arrExpIds/1E5).astype(int) * 1E5).astype(int)
fig, axs = plt.subplots(2, 5, figsize=(16, 5))
plt.subplots_adjust(wspace=0.8, hspace=0.6)
plt.suptitle(f"Compensation Offsets vs LUT {startDay}-{endDay}", fontsize=18, y=1.0)
for i in range(5):
    comps = [term[i] for term in camComps]
    lut = [term[i] for term in camLUTs]
    axs[0][i].scatter(lut, comps, marker='x')
    axs[0][i].set_title(f'Cam{names[i]}')
    if i < 3:
        axs[0][i].set_xlabel("Microns")
        axs[0][i].set_ylabel("Microns")
    else:
        axs[0][i].set_xlabel("Arcseconds")
        axs[0][i].set_ylabel("Arcseconds")
    comps = [term[i] for term in m2Comps]
    lut = [term[i] for term in m2LUTs]
    axs[1][i].scatter(lut, comps, marker='x')
    axs[1][i].set_title(f'M2{names[i]}')
    if i < 3:
        axs[1][i].set_xlabel("Microns")
        axs[1][i].set_ylabel("Microns")
    else:
        axs[1][i].set_xlabel("Arcseconds")
        axs[1][i].set_ylabel("Arcseconds")
plt.savefig(f"/home/c/cslage/u/Hexapods/LUTs/LUT_vs_CompOffset_{startDay}-{endDay}.png")
plt.clf


In [ ]:
def calculateHexMotion(camLUTs, m2LUTs):
    # The below image motion coefficients were calculated
    # with a Batoid simulation by Josh Meyers
    camHexXY = 1.00  # microns(image) / micron(hexapod)
    camHexUV = 4.92  # microns(image) / arcsecond(hexapod)
    m2HexXY = 1.13  # microns(image) / micron(hexapod)
    m2HexUV = 37.26  # microns(image) / arcsecond(hexapod)

    # Convert these to image impact in arcseconds
    # The 10.0 is microns / pixel
    # Still need to double check signs!!
    pixelScale = 0.2  # arcseconds / pixel - find this elsewhere?
    camHexXY = camHexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    camHexUV = camHexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    camCoefsX = [-camHexXY, 0, 0, 0, -camHexUV, 0]
    camCoefsY = [0, -camHexXY, 0, camHexUV, 0, 0]
    m2HexXY = m2HexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    m2HexUV = m2HexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    m2CoefsX = [-m2HexXY, 0, 0, 0, -m2HexUV, 0]
    m2CoefsY = [0, -m2HexXY, 0, m2HexUV, 0, 0]

    camLUT = [term[0] for term in camLUTs]
    shiftXs = np.zeros([len(camLUT), 10])
    shiftYs = np.zeros([len(camLUT), 10])
    for j in range(len(camLUT)):
        for i in range(5):
            camLUT = [term[i] for term in camLUTs]
            m2LUT = [term[i] for term in camLUTs]
            shiftXs[j, i] = camLUT[j] * camCoefsX[i]
            shiftYs[j, i] = camLUT[j] * camCoefsY[i]
            shiftXs[j, i+5] = m2LUT[j] * m2CoefsX[i]
            shiftYs[j, i+5] = m2LUT[j] * m2CoefsY[i]
    return [shiftXs, shiftYs]

In [ ]:
[shiftXs, shiftYs] = calculateHexMotion(camLUTs, m2LUTs)
plt.title(f"Total Hexapod LUT induced image shift - {startDay}")
plt.scatter(np.sum(shiftXs[0:240,:], axis=1), np.sum(shiftYs[0:240,:], axis=1), label='seqNum < 367')
plt.scatter(np.sum(shiftXs[240:-1,:], axis=1), np.sum(shiftYs[240:-1,:], axis=1), label='seqNum >= 367')
plt.xlabel("DeltaX (arcseconds)")
plt.ylabel("DeltaY (arcseconds)")
plt.legend()
plt.savefig(f"/home/c/cslage/u/Hexapods/LUTs/Total_Hexapod_Shift_{startDay}.png")


In [ ]:
[shiftXs, shiftYs] = calculateHexMotion(camLUTs, m2LUTs)
names = ['X', 'Y', 'Z', 'U', 'V']
arrExpIds = np.array(expIds)
seqNums = (arrExpIds - (arrExpIds/1E5).astype(int) * 1E5).astype(int)
fig, axs = plt.subplots(2, 4, figsize=(12, 5))
plt.subplots_adjust(wspace=0.8, hspace=0.6)
plt.suptitle(f"Hexapod induced image shifts {startDay}-{endDay}", fontsize=18, y=1.0)
axs[0][0].scatter(els[0:240], shiftXs[0:240,0])
axs[0][0].scatter(els[240:-1], shiftXs[240:-1,0])
axs[0][0].set_title(f'Cam{names[0]}')
axs[0][1].scatter(els[0:240], shiftYs[0:240,1])
axs[0][1].scatter(els[240:-1], shiftYs[240:-1,1])
axs[0][1].set_title(f'Cam{names[1]}')
axs[0][2].scatter(els[0:240], shiftYs[0:240,3])
axs[0][2].scatter(els[240:-1], shiftYs[240:-1,3])
axs[0][2].set_title(f'Cam{names[3]}')
axs[0][3].scatter(els[0:240], shiftXs[0:240,4])
axs[0][3].scatter(els[240:-1], shiftXs[240:-1,4])
axs[0][3].set_title(f'Cam{names[4]}')
axs[1][0].scatter(els[0:240], shiftXs[0:240,5])
axs[1][0].scatter(els[240:-1], shiftXs[240:-1,5])
axs[1][0].set_title(f'M2{names[0]}')
axs[1][1].scatter(els[0:240], shiftYs[0:240,6])
axs[1][1].scatter(els[240:-1], shiftYs[240:-1,6])
axs[1][1].set_title(f'M2{names[1]}')
axs[1][2].scatter(els[0:240], shiftYs[0:240,8])
axs[1][2].scatter(els[240:-1], shiftYs[240:-1,8])
axs[1][2].set_title(f'M2{names[3]}')
axs[1][3].scatter(els[0:240], shiftXs[0:240,9])
axs[1][3].scatter(els[240:-1], shiftXs[240:-1,9])
axs[1][3].set_title(f'M2{names[4]}')
for i in range(4):
    axs[0][i].set_xlabel("Elevation (degrees)")
    axs[0][i].set_ylabel("Arcseconds")
    axs[1][i].set_ylabel("Arcseconds")
#plt.savefig(f"/home/c/cslage/u/Hexapods/LUTs/LUT_vs_CompOffset_{startDay}-{endDay}.png")


In [ ]:
startDay = 20250826
endDay = 20250828

dayObs = startDay
expIds = []
xaxis = []
camPoss = []
camLUTs = []
m2Poss = []
m2LUTs = []
temps = []
counter = 1
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['acq', 'science']:
            continue
        try:
            temp = getEfdData(client, 'lsst.sal.ESS.temperature',
                 expRecord=record)
            m2Temp = temp[temp['sensorName']=='M2-ESS02']
            strutTemp = (np.nanmedian(m2Temp['temperatureItem6'].values) + \
                          np.nanmedian(m2Temp['temperatureItem7'].values)) / 2.0
            temps.append(strutTemp)  
            el = 90.0 - record.zenith_angle
            (mountErrors, mountData) = calculateMountErrors(record, client)
            rotVals = mountData.rotationData['actualPosition'].values
            rot = (np.median(rotVals))
            poss = []
            luts = []
            for i in range(5):
                val = np.median(mountData.camhexData[f"position{i}"].values)
                lut_coeffs = lut_data['camera_config']['elevation_coeffs'][i]
                rot_coeffs = lut_data['camera_config']['rotation_coeffs'][i]
                lut = np.polyval(lut_coeffs[::-1], el)
                lut += np.polyval(rot_coeffs[::-1], rot)
                if i in [3, 4]:
                    val *= 3600.0
                    lut *= 3600.0
                poss.append(val)
                luts.append(lut)
            camPoss.append(poss)
            camLUTs.append(luts)
            poss = []
            luts = []
            for i in range(5):
                val = np.median(mountData.m2hexData[f"position{i}"].values)
                lut_coeffs = lut_data['m2_config']['elevation_coeffs'][i]
                rot_coeffs = lut_data['m2_config']['rotation_coeffs'][i]
                lut = np.polyval(lut_coeffs[::-1], el)
                lut += np.polyval(rot_coeffs[::-1], rot)
                if i in [3, 4]:
                    val *= 3600.0
                    lut *= 3600.0
                poss.append(val)
                luts.append(lut)
            m2Poss.append(poss)
            m2LUTs.append(luts)
            xaxis.append(counter)
            expIds.append(record.id)
            counter += 1
            print(f"{record.id} succeeded!")
        except:
            print(f"{record.id} failed!")
            continue
    dayObs = calcNextDay(dayObs)


In [ ]:
dayObss = [20250826, 20250827, 20250828]


expIds = []
xaxis = []
camPoss = []
m2Poss = []
temps = []
counter = 1
for dayObs in dayObss:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['science'] or \
        record.science_program not in ['BLOCK-365']:
            continue
        try:
            temp = getEfdData(client, 'lsst.sal.ESS.temperature',
                 expRecord=record)
            m2Temp = temp[temp['sensorName']=='M2-ESS02']
            strutTemp = (np.nanmedian(m2Temp['temperatureItem6'].values) + \
                          np.nanmedian(m2Temp['temperatureItem7'].values)) / 2.0
            temps.append(strutTemp)  
            el = 90.0 - record.zenith_angle
            (mountErrors, mountData) = calculateMountErrors(record, client)
            rotVals = mountData.rotationData['actualPosition'].values
            rot = (np.median(rotVals))
            poss = []
            for i in range(5):
                val = np.median(mountData.camhexData[f"position{i}"].values)
                if i in [3, 4]:
                    val *= 3600.0
                poss.append(val)
            camPoss.append(poss)
            poss = []
            for i in range(5):
                val = np.median(mountData.m2hexData[f"position{i}"].values)
                if i in [3, 4]:
                    val *= 3600.0
                poss.append(val)
            m2Poss.append(poss)
            xaxis.append(counter)
            expIds.append(record.id)
            counter += 1
            print(f"{record.id} succeeded!")
        except:
            print(f"{record.id} failed!")
            continue


In [ ]:
dayObss = [20250826, 20250827, 20250828]

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
plt.suptitle(f"CamZ - M2Z vs Air Temp ", fontsize=12)
camZ = [term[2] for term in camPoss]
m2Z = [term[2] for term in m2Poss]
diff = np.array(camZ) - np.array(m2Z)
for j, dayObs in enumerate(dayObss):
    this_diff = [diff[i] for i in range(len(diff)) if (int(expIds[i] / 1E5) == dayObs)]
    this_temp = [temps[i] for i in range(len(diff)) if (int(expIds[i] / 1E5) == dayObs)]
    #print(dayObs, this_diff[10:15])
    #print(dayObs, this_temp[10:15])
    ax.scatter(this_temp, this_diff, marker='x', label=dayObs)
ax.legend()
ax.set_ylabel("Microns")
ax.set_xlabel("Strut Temp (Mean of TMA pylon +X/+Y and -X/-Y) (C)")
#ax.set_xlabel("Air Temp above M1M3 (C)")
plt.savefig(f"/home/c/cslage/u/Hexapods/LUTs/ZDiff_vs_Temp_Multi_SV.png")


In [ ]:
for i in range(len(expIds)):
    print(i, expIds[i])

In [ ]:
startDay = 20250909
endDay = 20250909

dayObs = startDay
expIds = []
xaxis = []
camPoss = []
camLUTs = []
m2Poss = []
m2LUTs = []
temps = []
counter = 1
cam_offset = [0,0,0,30,30]
m2_offset = [0,0,500,30,0]
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['acq', 'science']:
            continue
        try:
            """
            temp = getEfdData(client, 'lsst.sal.ESS.temperature',
                 expRecord=record)
            m2Temp = temp[temp['sensorName']=='M2-ESS02']
            strutTemp = (np.nanmedian(m2Temp['temperatureItem6'].values) + \
                          np.nanmedian(m2Temp['temperatureItem7'].values)) / 2.0
            temps.append(strutTemp) 
            """
            el = 90.0 - record.zenith_angle
            (mountErrors, mountData) = calculateMountErrors(record, client)
            rotVals = mountData.rotationData['actualPosition'].values
            rot = (np.median(rotVals))
            poss = []
            luts = []
            for i in range(5):
                val = np.median(mountData.camhexData[f"position{i}"].values)
                lut_coeffs = np.array(lut_data_4['camera_config']['elevation_rotation_coeffs'][i])
                lut = npp.polyval2d(el, rot, lut_coeffs)
                if i in [3, 4]:
                    val *= 3600.0
                    lut *= 3600.0
                #lut += cam_offset[i]
                poss.append(val)
                luts.append(lut)
            camPoss.append(poss)
            camLUTs.append(luts)
            poss = []
            luts = []
            for i in range(5):
                val = np.median(mountData.m2hexData[f"position{i}"].values)
                lut_coeffs = np.array(lut_data_4['m2_config']['elevation_rotation_coeffs'][i])
                lut = npp.polyval2d(el, rot, lut_coeffs)
                if i in [3, 4]:
                    val *= 3600.0
                    lut *= 3600.0
                #lut += m2_offset[i]
                poss.append(val)
                luts.append(lut)
            m2Poss.append(poss)
            m2LUTs.append(luts)
            xaxis.append(counter)
            expIds.append(record.id)
            counter += 1
            print(f"{record.id} succeeded!")
        except:
            print(f"{record.id} failed!")
            continue
    dayObs = calcNextDay(dayObs)


In [ ]:
dayObs = startDay
names = ['X', 'Y', 'Z', 'U', 'V']
arrExpIds = np.array(expIds)
seqNums = (arrExpIds - (arrExpIds/1E5).astype(int) * 1E5).astype(int)
fig, axs = plt.subplots(2, 5, figsize=(16, 5))
plt.subplots_adjust(wspace=0.8, hspace=0.6)
plt.suptitle(f"Hexapod changes vs LUT {startDay}-{endDay}", fontsize=18, y=1.0)
for i in range(5):
    cam = [term[i] for term in camPoss]
    lut = [term[i] for term in camLUTs]
    axs[0][i].scatter(xaxis, cam, marker='x')
    axs[0][i].scatter(xaxis, lut, marker='.', color='red')
    axs[0][i].set_xticks(xaxis[::50])
    axs[0][i].set_xticklabels(seqNums[::50], fontsize=10)
    axs[0][i].tick_params(axis='x', labelrotation=90)
    axs[0][i].set_title(f'Cam{names[i]}')
    #axs[0][i].axvline(239.5, ls='--', color='green')
    if i < 3:
        axs[0][i].set_ylabel("Microns")
    else:
        axs[0][i].set_ylabel("Arcseconds")
    m2 = [term[i] for term in m2Poss]
    lut = [term[i] for term in m2LUTs]
    axs[1][i].scatter(xaxis, m2, marker='x')
    axs[1][i].scatter(xaxis, lut, marker='.', color='red')
    axs[1][i].set_xticks(xaxis[::50])
    axs[1][i].set_xticklabels(seqNums[::50], fontsize=10)
    axs[1][i].tick_params(axis='x', labelrotation=90)
    axs[1][i].set_title(f'M2{names[i]}')
    #axs[1][i].axvline(239.5, ls='--', color='green')
    if i < 3:
        axs[1][i].set_ylabel("Microns")
    else:
        axs[1][i].set_ylabel("Arcseconds")
plt.savefig(f"/home/c/cslage/u/Hexapods/LUTs/Hexapod_Changes_LUT_{startDay}.png")
plt.clf


In [ ]:
expId = 2025090900104
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)
dof = getMostRecentRowWithDataBefore(client, "lsst.sal.MTAOS.logevent_degreeOfFreedom", \
                                                    expRecord.timespan.end)
len(dof)

In [ ]:
for i in range(10):
    print(i, dof[f"aggregatedDoF{i}"])

In [ ]:
def CamSalIndexTest(df):
    return df['salIndex'] == 1
def M2SalIndexTest(df):
    return df['salIndex'] == 2

expId = 2025090900104
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)
comp = getMostRecentRowWithDataBefore(client, "lsst.sal.MTHexapod.logevent_compensationOffset", \
                                                    expRecord.timespan.end, where=CamSalIndexTest)
len(comp)

In [ ]:
comp

In [ ]:
lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/laser_rotation_elevation_v20_positive.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)


In [ ]:
el = np.linspace(20, 85, 14)
#for r in [-45, 0, 45]:
for r in [0.142, 2.34]:
    rot = np.linspace(r, r, 14)
    i = 0
    lut_coeffs = lut_coeffs = np.array(lut_data['camera_config']['elevation_rotation_coeffs'][i])
    lut = npp.polyval2d(el, rot, lut_coeffs)
    plt.plot(el, lut, marker='x', label=f"Rot={r:.0f}")
plt.legend()

In [ ]:
startDay = 20251103
endDay = 20251103
firstImage = 10
lastImage = 421
dayObs = startDay
expIds = []
xaxis = []
camPoss = []
camLUTs = []
camDofs = []
m2Poss = []
m2LUTs = []
m2Dofs = []
temps = []
counter = 1
m2DoFIndex = [1,2,0,3,4]
camDoFIndex = [6,7,5,8,9]

while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        #if record.observation_type not in ['acq', 'science']:
        #    continue
        if (id > int(dayObs*1e5 + lastImage)) or (id < int(dayObs*1E5 + firstImage)):
            continue
        try:
            el = 90.0 - record.zenith_angle
            dof = getMostRecentRowWithDataBefore(client, \
                         "lsst.sal.MTAOS.logevent_degreeOfFreedom", \
                         record.timespan.end)
            if record.seq_num == 50:
                this_dof = dof

            (mountErrors, mountData) = calculateMountErrors(record, client)
            rotVals = mountData.rotationData['actualPosition'].values
            rot = (np.median(rotVals))
            poss = []
            luts = []
            dofs = []
            for i in range(5):
                val = np.median(mountData.camhexData[f"position{i}"].values)
                lut_coeffs = np.array(lut_data['camera_config']['elevation_rotation_coeffs'][i])
                lut = npp.polyval2d(el, rot, lut_coeffs)
                #if i == 0:
                #    print(el, rot, lut)
                    
                dofs.append(dof[f"aggregatedDoF{camDoFIndex[i]}"])
                if i in [3, 4]:
                    val *= 3600.0
                    lut *= 3600.0
                poss.append(val)
                luts.append(lut)
            camPoss.append(poss)
            camLUTs.append(luts)
            camDofs.append(dofs)
            poss = []
            luts = []
            dofs = []
            for i in range(5):
                val = np.median(mountData.m2hexData[f"position{i}"].values)
                lut_coeffs = np.array(lut_data['m2_config']['elevation_rotation_coeffs'][i])
                lut = npp.polyval2d(el, rot, lut_coeffs)
                dofs.append(dof[f"aggregatedDoF{m2DoFIndex[i]}"])
                if i in [3, 4]:
                    val *= 3600.0
                    lut *= 3600.0
                poss.append(val)
                luts.append(lut)
            m2Poss.append(poss)
            m2LUTs.append(luts)
            m2Dofs.append(dofs)
            xaxis.append(record.seq_num)
            expIds.append(record.id)
            counter += 1
            print(f"{record.id} succeeded!")
        except:
            print(f"{record.id} failed!")
            continue
    dayObs = calcNextDay(dayObs)


In [ ]:
len(xaxis)

In [ ]:
dayObs = startDay
names = ['X', 'Y', 'Z', 'U', 'V']
arrExpIds = np.array(expIds)
fig, axs = plt.subplots(2, 5, figsize=(16, 5))
plt.subplots_adjust(wspace=0.8, hspace=0.6)
plt.suptitle(f"Hexapod changes vs LUT {startDay}-{endDay}", fontsize=18, y=1.0)
for i in range(5):
    cam = [term[i] for term in camPoss]
    lut = [term[i] for term in camLUTs]
    dof = [term[i] for term in camDofs]
    axs[0][i].scatter(xaxis, cam, marker='x')
    axs[0][i].scatter(xaxis, lut, marker='.', color='red')
    axs[0][i].scatter(xaxis, dof, marker='.', color='green')
    #axs[0][i].set_xticks(xaxis[::50])
    #axs[0][i].set_xticklabels(seqNums[::50], fontsize=10)
    axs[0][i].tick_params(axis='x', labelrotation=90)
    axs[0][i].set_title(f'Cam{names[i]}')
    #axs[0][i].axvline(239.5, ls='--', color='green')
    if i < 3:
        axs[0][i].set_ylabel("Microns")
    else:
        axs[0][i].set_ylabel("Arcseconds")
    m2 = [term[i] for term in m2Poss]
    lut = [term[i] for term in m2LUTs]
    dof = [term[i] for term in m2Dofs]
    axs[1][i].scatter(xaxis, m2, marker='x')
    axs[1][i].scatter(xaxis, lut, marker='.', color='red')
    axs[1][i].scatter(xaxis, dof, marker='.', color='green')
    #axs[1][i].set_xticks(xaxis[::50])
    #axs[1][i].set_xticklabels(seqNums[::50], fontsize=10)
    axs[1][i].tick_params(axis='x', labelrotation=90)
    axs[1][i].set_title(f'M2{names[i]}')
    #axs[1][i].axvline(239.5, ls='--', color='green')
    if i < 3:
        axs[1][i].set_ylabel("Microns")
    else:
        axs[1][i].set_ylabel("Arcseconds")
plt.savefig(f"/home/c/cslage/u/Hexapods/LUTs/Hexapod_Changes_LUT_DOF_{startDay}.png")
plt.clf


In [ ]:
dayObs = startDay
names = ['X', 'Y', 'Z', 'U', 'V']
arrExpIds = np.array(expIds)
fig, axs = plt.subplots(2, 5, figsize=(16, 5))
plt.subplots_adjust(wspace=0.8, hspace=0.6)
plt.suptitle(f"Hexapod changes vs LUT {startDay}-{endDay}", fontsize=18, y=1.0)
for i in range(5):
    cam = [term[i] for term in camPoss]
    lut = [term[i] for term in camLUTs]
    dof = [term[i] for term in camDofs]
    axs[0][i].scatter(xaxis, cam, marker='x')
    #axs[0][i].scatter(xaxis, lut, marker='.', color='red')
    axs[0][i].scatter(xaxis, dof, marker='.', color='green')

    axs[0][i].tick_params(axis='x', labelrotation=90)
    axs[0][i].set_title(f'Cam{names[i]}')
    if i < 3:
        axs[0][i].set_ylabel("Microns")
    else:
        axs[0][i].set_ylabel("Arcseconds")
    m2 = [term[i] for term in m2Poss]
    lut = [term[i] for term in m2LUTs]
    dof = [term[i] for term in m2Dofs]
    axs[1][i].scatter(xaxis, m2, marker='x')
    #axs[1][i].scatter(xaxis, lut, marker='.', color='red')
    axs[1][i].scatter(xaxis, dof, marker='.', color='green')
    
    axs[1][i].tick_params(axis='x', labelrotation=90)
    axs[1][i].set_title(f'M2{names[i]}')
    if i < 3:
        axs[1][i].set_ylabel("Microns")
    else:
        axs[1][i].set_ylabel("Arcseconds")
plt.savefig(f"/home/c/cslage/u/Hexapods/LUTs/Hexapod_Changes_LUT_DOF_{startDay}.png")
plt.clf


In [ ]:
this_dof[f"aggregatedDoF{camDoFIndex[2]}"]

In [ ]:
this_dof["kpGain"]

In [ ]:
i = 0
dof = [term[i] for term in m2Dofs]
dof